In [1]:
from sklearn import metrics
import processing
import time
import os
from tqdm import tqdm
from torch.optim import Adam
from torch import nn
import numpy as np
import sys
sys.path.append("D:/Experiment")
import torch
from MyKu import processing
from MyKu import MyXLM_TW
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 训练准备阶段，设置超参数和全局变量

batch_size = 32
num_epoch = 20  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
en_train_data, en_test_data, de_train_data, de_test_data, hi_train_data, hi_test_data = processing.load_hasoc2020_for_taskB()
categories = 3

en_train_iter, en_test_iter = MyXLM_TW.load_xlm_data(en_train_data, en_test_data, batch_size)
de_train_iter, de_test_iter = MyXLM_TW.load_xlm_data(de_train_data, de_test_data, batch_size)
hi_train_iter, hi_test_iter = MyXLM_TW.load_xlm_data(hi_train_data, hi_test_data, batch_size)
#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
# 创建模型 BertSST2Model
model = MyXLM_TW.MyXlmModel(categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())

In [6]:

file_name = 'readme_tw_taskb.md'
model_save_path = 'D:/Experiment_models_save/twitter-xlm-roberta-base-sentiment/hasoc2020/'
name = 'tw_taskB.pth'
en_temp_best = 0.78
de_temp_best = 0.74
hi_temp_best = 0.58
for epoch in range(1, num_epoch + 1):
    MyXLM_TW.train(model, en_train_iter, device, optimizer, loss, epoch)
    MyXLM_TW.train(model, de_train_iter, device, optimizer, loss, epoch)
    MyXLM_TW.train(model, hi_train_iter, device, optimizer, loss, epoch)
    en_acc_score = MyXLM_TW.test(model, en_test_iter, device, epoch, file_name)
    de_acc_score = MyXLM_TW.test(model, de_test_iter, device, epoch, file_name)
    hi_acc_score = MyXLM_TW.test(model, hi_test_iter, device, epoch, file_name)
    print('\n\n')
    if en_acc_score > en_temp_best and de_acc_score > de_temp_best and hi_acc_score > hi_temp_best:
        en_temp_best, de_temp_best, hi_temp_best = en_acc_score, de_acc_score, hi_acc_score
        MyXLM_TW.save_pretrained(model, model_save_path, name)
        print(
            f'best en_acc_socre : {en_acc_score}, best de_acc_score : {de_acc_score}, best hi_acc_score : {hi_acc_score}')
print(
    f'best en_acc_socre : {en_temp_best}, best de_acc_score : {de_temp_best}, best hi_acc_score : {hi_acc_score}')


Testing: 100%|██████████| 807/807 [00:10<00:00, 79.94it/s]


[[538  21  21]
 [ 99  56  14]
 [ 32   9  17]]
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       580
           1       0.65      0.33      0.44       169
           2       0.33      0.29      0.31        58

    accuracy                           0.76       807
   macro avg       0.59      0.52      0.54       807
weighted avg       0.74      0.76      0.73       807

Acc : 0.7571251548946716	 F1: 0.5365985955728337


Testing: 100%|██████████| 300/300 [00:04<00:00, 73.00it/s]


[[163   8   6]
 [ 33  27   6]
 [  6   4  47]]
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       177
           1       0.69      0.41      0.51        66
           2       0.80      0.82      0.81        57

    accuracy                           0.79       300
   macro avg       0.77      0.72      0.73       300
weighted avg       0.78      0.79      0.77       300

Acc : 0.79	 F1: 0.7282629510725225


Testing: 100%|██████████| 367/367 [00:05<00:00, 70.61it/s]


...

[[541  23  16]
 [109  51   9]
 [ 27  12  19]]
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       580
           1       0.59      0.30      0.40       169
           2       0.43      0.33      0.37        58

    accuracy                           0.76       807
   macro avg       0.61      0.52      0.54       807
weighted avg       0.73      0.76      0.73       807

Acc : 0.7571251548946716	 F1: 0.5444428845523891


Testing: 100%|██████████| 300/300 [00:02<00:00, 102.92it/s]


[[154  15   8]
 [ 30  27   9]
 [  6   6  45]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       177
           1       0.56      0.41      0.47        66
           2       0.73      0.79      0.76        57

    accuracy                           0.75       300
   macro avg       0.70      0.69      0.69       300
weighted avg       0.74      0.75      0.74       300

Acc : 0.7533333333333333	 F1: 0.6897412629184757


Testing: 100%|██████████| 367/367 [00:03<00:00, 100.45it/s]

[[ 28  33   2]
 [ 19 140  32]
 [  6  49  58]]
              precision    recall  f1-score   support

           0       0.53      0.44      0.48        63
           1       0.63      0.73      0.68       191
           2       0.63      0.51      0.57       113

    accuracy                           0.62       367
   macro avg       0.60      0.56      0.58       367
weighted avg       0.61      0.62      0.61       367

Acc : 0.6158038147138964	 F1: 0.5755261269737186



best en_acc_socre : 0.78, best de_acc_score : 0.74, best hi_acc_score : 0.6158038147138964


In [ ]:
from transformers import XLMRobertaForSequenceClassification

xlm = XLMRobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-xlm-roberta-base-sentiment', return_dict=True)
xlm.classifier.out_proj = nn.Linear(768, 2)
print(xlm.eval())